<a href="https://colab.research.google.com/github/Mustow/WhatsApp_Reel_AutoSplitter/blob/main/WhatsApp_Reel_Video_Splitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
WhatsApp Reel Video Splitter - Google Colab Backend (No Re-encoding)
Fast splitting using FFmpeg stream copy - preserves original quality
"""

# Install ffmpeg-python for direct FFmpeg control
!pip install ffmpeg-python -q

import os
import subprocess
import math
from google.colab import files
import json
import zipfile
import shutil

class WhatsAppReelSplitter:
    def __init__(self, split_duration=30):
        """
        Initialize the video splitter

        Args:
            split_duration: Duration per clip in seconds (default: 30)
        """
        self.split_duration = split_duration
        self.output_folder = "whatsapp_reels"

    def create_output_folder(self):
        """Create output folder for split videos"""
        if os.path.exists(self.output_folder):
            shutil.rmtree(self.output_folder)
        os.makedirs(self.output_folder)

    def get_video_duration(self, video_path):
        """Get video duration using ffprobe"""
        cmd = [
            'ffprobe',
            '-v', 'error',
            '-show_entries', 'format=duration',
            '-of', 'json',
            video_path
        ]

        result = subprocess.run(cmd, capture_output=True, text=True)
        try:
            data = json.loads(result.stdout)
            duration = float(data['format']['duration'])
            return duration
        except (json.JSONDecodeError, KeyError):
            print(f"Error getting duration for {video_path}")
            return 0

    def get_video_info(self, video_path):
        """Get detailed video information using ffprobe"""
        cmd = [
            'ffprobe',
            '-v', 'error',
            '-show_entries', 'format=size,duration:stream=width,height,codec_name,bit_rate',
            '-of', 'json',
            video_path
        ]

        result = subprocess.run(cmd, capture_output=True, text=True)
        try:
            data = json.loads(result.stdout)

            file_size_mb = os.path.getsize(video_path) / (1024 * 1024)
            duration = float(data['format'].get('duration', 0)) # Use .get with default 0

            # Use .get with default [] for streams and .get with default None for codec_type
            video_stream = next((s for s in data.get('streams', []) if s.get('codec_type') == 'video'), None)


            info = {
                'duration': duration,
                'file_size_mb': file_size_mb,
                'width': video_stream.get('width', 'N/A') if video_stream else 'N/A',
                'height': video_stream.get('height', 'N/A') if video_stream else 'N/A',
                'codec': video_stream.get('codec_name', 'N/A') if video_stream else 'N/A'
            }

            return info
        except (json.JSONDecodeError, KeyError) as e:
            print(f"Error getting video info for {video_path}: {e}")
            return {
                'duration': 0,
                'file_size_mb': 0,
                'width': 'N/A',
                'height': 'N/A',
                'codec': 'N/A'
            }


    def split_video(self, video_path):
        """
        Split video using FFmpeg stream copy (no re-encoding)
        This preserves original quality and is MUCH faster
        """
        self.create_output_folder()

        # Get video duration
        total_duration = self.get_video_duration(video_path)

        if total_duration == 0:
            print("❌ Could not get video duration. Aborting split.")
            return []


        print(f"\n📹 Video Duration: {total_duration:.2f} seconds")
        print(f"✂️ Split Interval: {self.split_duration} seconds")

        # Calculate number of clips
        num_clips = math.ceil(total_duration / self.split_duration)
        print(f"📦 Will create: {num_clips} clips\n")

        clips_info = []

        for i in range(num_clips):
            start_time = i * self.split_duration

            # Calculate duration for this clip
            if i == num_clips - 1:
                # Last clip - take remaining duration
                clip_duration = total_duration - start_time
            else:
                clip_duration = self.split_duration

            output_path = os.path.join(self.output_folder, f"reel_{i+1:02d}.mp4")

            print(f"⚡ Processing clip {i+1}/{num_clips}...")

            # FFmpeg command with stream copy (no re-encoding)
            cmd = [
                'ffmpeg',
                '-ss', str(start_time),           # Start time
                '-i', video_path,                  # Input file
                '-t', str(clip_duration),          # Duration
                '-c', 'copy',                      # Copy streams (no re-encoding)
                '-avoid_negative_ts', '1',         # Fix timestamp issues
                '-y',                              # Overwrite output
                output_path
            ]

            # Run FFmpeg
            result = subprocess.run(
                cmd,
                capture_output=True,
                text=True
            )

            if result.returncode != 0:
                print(f"⚠️ Warning: Clip {i+1} had issues")
                print(result.stderr)

            # Get output file info
            if os.path.exists(output_path):
                file_size_mb = os.path.getsize(output_path) / (1024 * 1024)

                clips_info.append({
                    'number': i + 1,
                    'filename': f"reel_{i+1:02d}.mp4",
                    'start': start_time,
                    'end': start_time + clip_duration,
                    'duration': clip_duration,
                    'size_mb': file_size_mb
                })

                print(f"✅ Clip {i+1}: {start_time:.1f}s - {start_time + clip_duration:.1f}s ({file_size_mb:.2f}MB)\n")
            else:
                print(f"❌ Failed to create clip {i+1}\n")

        return clips_info

    def create_zip(self):
        """Create a zip file of all split videos"""
        zip_path = "whatsapp_reels.zip"
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for file in os.listdir(self.output_folder):
                if file.endswith('.mp4'):
                    file_path = os.path.join(self.output_folder, file)
                    zipf.write(file_path, file)
        return zip_path


def split_video_for_whatsapp(split_duration=30):
    """
    Main function to split videos for WhatsApp
    Uses FFmpeg stream copy for fast, lossless splitting

    Args:
        split_duration: Seconds per clip (default: 30)
    """
    print("=" * 60)
    print("🎥 WHATSAPP REEL VIDEO SPLITTER (NO RE-ENCODING)")
    print("=" * 60)

    # Upload video
    print("\n📤 Upload your video file:")
    uploaded = files.upload()

    if not uploaded:
        print("❌ No file uploaded!")
        return

    video_path = list(uploaded.keys())[0]
    print(f"\n✅ Uploaded: {video_path}")

    # Initialize splitter
    splitter = WhatsAppReelSplitter(split_duration=split_duration)

    # Get video info
    print("\n📊 Analyzing video...")
    info = splitter.get_video_info(video_path)
    print(f"   Duration: {info['duration']:.2f}s")
    print(f"   Resolution: {info['width']}x{info['height']}")
    print(f"   Codec: {info['codec']}")
    print(f"   File Size: {info['file_size_mb']:.2f}MB")

    # Split video (NO RE-ENCODING)
    print("\n" + "=" * 60)
    print("⚡ SPLITTING VIDEO (Stream Copy - No Re-encoding)")
    print("=" * 60)

    clips_info = splitter.split_video(video_path)

    if not clips_info:
        print("❌ No clips were created!")
        return

    # Create summary
    print("=" * 60)
    print("✅ SPLITTING COMPLETE!")
    print("=" * 60)
    print(f"\n📦 Created {len(clips_info)} clips:")

    total_size = 0
    for clip in clips_info:
        print(f"   {clip['filename']}: {clip['duration']:.1f}s ({clip['size_mb']:.2f}MB)")
        total_size += clip['size_mb']

    print(f"\n💾 Total size: {total_size:.2f}MB")

    # Create zip and download
    print("\n📦 Creating zip file...")
    zip_path = splitter.create_zip()
    zip_size = os.path.getsize(zip_path) / (1024 * 1024)
    print(f"✅ Zip created: {zip_path} ({zip_size:.2f}MB)")

    print("\n⬇️ Downloading zip file...")
    files.download(zip_path)

    print("\n🎉 Done! Your WhatsApp reels are ready!")
    print("\n💡 Tips:")
    print("   • Videos are NOT re-encoded (original quality preserved)")
    print("   • Splitting is super fast (seconds, not minutes)")
    print("   • If clips are too large, you may need to compress the original")

    return clips_info


# ============================================
# USAGE
# ============================================

print("""
╔════════════════════════════════════════════════════════════╗
║    WHATSAPP REEL VIDEO SPLITTER - NO RE-ENCODING!         ║
╚════════════════════════════════════════════════════════════╝

📝 INSTRUCTIONS:
1. Run the command below to start:

   split_video_for_whatsapp(split_duration=30)

2. Upload your video when prompted
3. Wait for splitting (FAST - no re-encoding!)
4. Download the zip file with all clips

⚡ FEATURES:
   ✅ NO re-encoding (preserves original quality)
   ✅ SUPER FAST (10x-100x faster than re-encoding)
   ✅ NO speed-up issues
   ✅ NO quality loss
   ✅ Exact 30-second intervals

🎯 CUSTOMIZATION:
   • split_video_for_whatsapp(split_duration=30)  → 30s clips
   • split_video_for_whatsapp(split_duration=15)  → 15s clips
   • split_video_for_whatsapp(split_duration=60)  → 60s clips

⚠️ NOTE:
   Since we're NOT re-encoding, file sizes stay proportional.
   If original video is 100MB and 120s long, each 30s clip ≈ 25MB.
   WhatsApp limit is 16MB, so compress original video first if needed.

🚀 Ready? Run: split_video_for_whatsapp(split_duration=30)
""")


╔════════════════════════════════════════════════════════════╗
║    WHATSAPP REEL VIDEO SPLITTER - NO RE-ENCODING!         ║
╚════════════════════════════════════════════════════════════╝

📝 INSTRUCTIONS:
1. Run the command below to start:

   split_video_for_whatsapp(split_duration=30)

2. Upload your video when prompted
3. Wait for splitting (FAST - no re-encoding!)
4. Download the zip file with all clips

⚡ FEATURES:
   ✅ NO re-encoding (preserves original quality)
   ✅ SUPER FAST (10x-100x faster than re-encoding)
   ✅ NO speed-up issues
   ✅ NO quality loss
   ✅ Exact 30-second intervals

🎯 CUSTOMIZATION:
   • split_video_for_whatsapp(split_duration=30)  → 30s clips
   • split_video_for_whatsapp(split_duration=15)  → 15s clips
   • split_video_for_whatsapp(split_duration=60)  → 60s clips

⚠️ NOTE:
   Since we're NOT re-encoding, file sizes stay proportional.
   If original video is 100MB and 120s long, each 30s clip ≈ 25MB.
   WhatsApp limit is 16MB, so compress original video fi

In [2]:
split_video_for_whatsapp(split_duration=30)

🎥 WHATSAPP REEL VIDEO SPLITTER (NO RE-ENCODING)

📤 Upload your video file:


Saving WIN_20250227_12_27_04_Pro.mp4 to WIN_20250227_12_27_04_Pro.mp4

✅ Uploaded: WIN_20250227_12_27_04_Pro.mp4

📊 Analyzing video...
   Duration: 72.75s
   Resolution: N/AxN/A
   Codec: N/A
   File Size: 70.43MB

⚡ SPLITTING VIDEO (Stream Copy - No Re-encoding)

📹 Video Duration: 72.75 seconds
✂️ Split Interval: 30 seconds
📦 Will create: 3 clips

⚡ Processing clip 1/3...
✅ Clip 1: 0.0s - 30.0s (29.31MB)

⚡ Processing clip 2/3...
✅ Clip 2: 30.0s - 60.0s (29.68MB)

⚡ Processing clip 3/3...
✅ Clip 3: 60.0s - 72.8s (12.83MB)

✅ SPLITTING COMPLETE!

📦 Created 3 clips:
   reel_01.mp4: 30.0s (29.31MB)
   reel_02.mp4: 30.0s (29.68MB)
   reel_03.mp4: 12.8s (12.83MB)

💾 Total size: 71.82MB

📦 Creating zip file...
✅ Zip created: whatsapp_reels.zip (71.73MB)

⬇️ Downloading zip file...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 Done! Your WhatsApp reels are ready!

💡 Tips:
   • Videos are NOT re-encoded (original quality preserved)
   • Splitting is super fast (seconds, not minutes)
   • If clips are too large, you may need to compress the original


[{'number': 1,
  'filename': 'reel_01.mp4',
  'start': 0,
  'end': 30,
  'duration': 30,
  'size_mb': 29.305986404418945},
 {'number': 2,
  'filename': 'reel_02.mp4',
  'start': 30,
  'end': 60,
  'duration': 30,
  'size_mb': 29.682636260986328},
 {'number': 3,
  'filename': 'reel_03.mp4',
  'start': 60,
  'end': 72.753367,
  'duration': 12.753366999999997,
  'size_mb': 12.82884693145752}]